# 制約された出力サンプル
文法規則 CFG に基づいて逐次生成テキストを解析し、制約を加える処理のテスト

In [ ]:
!pip install outlines

In [ ]:
from outlines.fsm.guide import CFGGuide, Generate, RegexGuide, StopAtEOSGuide, Write

class MockTokenizer:
    # モックのトークナイザーを定義します。簡略化された辞書とトークン変換メソッドを持っています。
    vocabulary = {"(": 1, ")": 2, "7": 3, "]": 4, "eos": 5}
    special_tokens = {"eos"}
    eos_token = "eos"
    eos_token_id = 5

    def convert_token_to_string(self, token):
        return token  # トークンを文字列に変換するメソッド（ここではそのまま返します）

    @property
    def inverse_vocabulary(self):
        return {v: k for k, v in self.vocabulary.items()}  # 辞書のキーと値を逆転させたものを返します

    def decode(self, token_ids):
        return [self.inverse_vocabulary[t] for t in token_ids]  # トークンIDを対応する文字列に変換します

cfg_str = """
start: expr

expr: "(" NUMBER ")"
NUMBER: /[0-9]+/
"""
# CFG (Context-Free Grammar) の定義。単純な文法で、数字を括弧で囲んで生成します。

tokenizer = MockTokenizer()  # 上記で定義したMockTokenizerをインスタンス化します
fsm = CFGGuide(cfg_str, tokenizer)  # CFGとトークナイザーを使ってFSM (Finite State Machine) を作成します


## 1 文字目の提案

In [ ]:
state = fsm.start_state  # FSM の初期状態を取得
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
tokenizer.decode(instruction.tokens)  # トークンIDを文字列に変換します

## 状態遷移①

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=1)
print(state, fsm.generation)

## 2 文字目の提案

In [ ]:
instruction = fsm.get_next_instruction(state)
print(instruction)
assert isinstance(instruction, Generate)

## 状態遷移②

In [ ]:
# 実際の値を投入して、次の状態へ遷移させる
state = fsm.get_next_state(state=state, token_id=3)
print(state, fsm.generation)

## 3 文字目の提案

In [ ]:
instruction = fsm.get_next_instruction(state)
print(instruction)
assert isinstance(instruction, Generate)

## 状態遷移③

In [ ]:
# 実際の値を投入して、次の状態へ遷移させる
state = fsm.get_next_state(state=state, token_id=3)
print(state, fsm.generation)

## n 文字目の提案

In [ ]:
instruction = fsm.get_next_instruction(state)
print(instruction)
assert isinstance(instruction, Generate)

In [ ]:
state = fsm.get_next_state(state=state, token_id=3)
print(state, fsm.generation)

In [ ]:
instruction = fsm.get_next_instruction(state)
print(instruction)
assert isinstance(instruction, Generate)

## 状態遷移n

In [ ]:
state = fsm.get_next_state(state=state, token_id=2)
print(state, fsm.generation)

## 終端文字の提案

In [ ]:
instruction = fsm.get_next_instruction(state)
print(instruction)
assert isinstance(instruction, Write)

In [ ]:
state = fsm.get_next_state(state=state, token_id=5)
print(state, fsm.generation)

In [ ]:
assert fsm.is_final_state(state)

# JSON パースの例

In [ ]:
class MockTokenizer:
    vocabulary = {"{": 1, "}": 2, "[": 3, "]": 4, ":": 5, ",": 6, "EOS": 7, "\"key\"": 8, "\"value\"": 9}
    special_tokens = {"EOS"}
    eos_token = "EOS"
    eos_token_id = 7

    def convert_token_to_string(self, token):
        return token

    @property
    def inverse_vocabulary(self):
        return {v: k for k, v in self.vocabulary.items()}

    def decode(self, token_ids):
        return [self.inverse_vocabulary[t] for t in token_ids]

# ユーザーからのJSONスキーマの入力
cfg_str = """
?start: value

?value: object
| array
| UNESCAPED_STRING
| SIGNED_NUMBER      -> number
| "true"             -> true
| "false"            -> false
| "null"             -> null

array  : "[" [value ("," value)*] "]"
object : "{" [pair ("," pair)*] "}"
pair   : UNESCAPED_STRING ":" value

%import common.UNESCAPED_STRING
%import common.SIGNED_NUMBER
%import common.WS

%ignore WS
"""

# トークナイザーインスタンス
tokenizer = MockTokenizer()
fsm = CFGGuide(cfg_str, tokenizer)


## 1 文字目の提案
token_id: [1, 9, 8, 3] が提案される。

In [ ]:
state = fsm.start_state  # FSM の初期状態を取得
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
tokenizer.decode(instruction.tokens)  # トークンIDを文字列に変換します

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=1)
print(state, fsm.generation)

In [ ]:
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
tokenizer.decode(instruction.tokens)

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=8)
print(state, fsm.generation)

In [ ]:
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=5)
print(state, fsm.generation)

In [ ]:
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=9)
print(state, fsm.generation)

In [ ]:
instruction = fsm.get_next_instruction(state)  # 次の指示を取得します
print(instruction) 

In [ ]:
state = fsm.get_next_state(state=fsm.start_state, token_id=2)
print(state, fsm.generation)